In [2]:
import pandas as pd
import sqlite3 as sql

pd.set_option('display.max_colwidth', None)

In [3]:
connection = sql.connect('data/sql-murder-mystery.db')

crsr = connection.cursor()


In [4]:
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [18]:
sql_query('Select * from crime_scene_report where date=20180115 and city="SQL City" and type="murder"')

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [20]:
sql_query('Select * from person where name like "%Annabel%" and address_street_name="Franklin Ave"')

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [21]:
sql_query('Select * from person where address_street_name="Northwestern Dr" and address_number=(Select max(address_number) from person where address_street_name="Northwestern Dr")')

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [22]:
sql_query('Select * from interview where person_id=16371 or person_id=14887')

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [8]:
sql_query('Select * from get_fit_now_member, drivers_license, get_fit_now_check_in where get_fit_now_check_in.check_in_date=20180109 and get_fit_now_member.id like "%48Z%" and get_fit_now_member.membership_status="gold" and drivers_license.plate_number like "%H42W%" and drivers_license.gender="male" and get_fit_now_check_in.membership_id=get_fit_now_member.id')

,id,person_id,name,membership_start_date,membership_status,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,664760,21,71,black,black,male,4H42WR,Nissan,Altima,48Z7A,20180109,1600,1730
1,48Z7A,28819,Joe Germuska,20160305,gold,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,48Z7A,20180109,1600,1730
2,48Z55,67318,Jeremy Bowers,20160101,gold,664760,21,71,black,black,male,4H42WR,Nissan,Altima,48Z55,20180109,1530,1700
3,48Z55,67318,Jeremy Bowers,20160101,gold,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,48Z55,20180109,1530,1700


In [26]:
sql_query('Select * from facebook_event_checkin where (person_id=28819 or person_id=67318)')

,person_id,event_id,event_name,date
0,67318,4719,The Funky Grooves Tour,20180115
1,67318,1143,SQL Symphony Concert,20171206


In [31]:
sql_query('Select * from interview where person_id=67318 or person_id=28819')

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [61]:
sql_query('Select count(*) from drivers_license, facebook_event_checkin, income where gender="female" and hair_color="red" and (height between 65 and 67) and car_make="Tesla" and car_model="Model S" and facebook_event_checkin.event_name="SQL Symphony Concert" and facebook_event_checkin.date like "%201712%" Group by person_id')

,count(*)
0,22542
1,22542
2,22542
3,22542
4,67626
5,45084
6,22542
7,22542
8,22542
9,22542


In [58]:
sql_query('Select * from income, person where person.id=202298 or person.id=11173')

,ssn,annual_income,id,name,license_id,address_number,address_street_name,ssn
0,100009868,52200,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569
1,100169584,64500,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569
2,100300433,74400,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569
3,100355733,35900,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569
4,100366269,73000,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569
...,...,...,...,...,...,...,...,...
7509,999679296,54400,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569
7510,999762859,77000,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569
7511,999824984,82000,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569
7512,999910617,82600,11173,Ken Whitted,755830,3300,Climbhill Rd,604322569


In [19]:
sql_query('Select * from income, person where person.id=28819')

,ssn,annual_income,id,name,license_id,address_number,address_street_name,ssn
0,100009868,52200,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,100169584,64500,28819,Joe Germuska,173289,111,Fisk Rd,138909730
2,100300433,74400,28819,Joe Germuska,173289,111,Fisk Rd,138909730
3,100355733,35900,28819,Joe Germuska,173289,111,Fisk Rd,138909730
4,100366269,73000,28819,Joe Germuska,173289,111,Fisk Rd,138909730
...,...,...,...,...,...,...,...,...
7509,999679296,54400,28819,Joe Germuska,173289,111,Fisk Rd,138909730
7510,999762859,77000,28819,Joe Germuska,173289,111,Fisk Rd,138909730
7511,999824984,82000,28819,Joe Germuska,173289,111,Fisk Rd,138909730
7512,999910617,82600,28819,Joe Germuska,173289,111,Fisk Rd,138909730


In [20]:
sql_query('Select * from income, person where person.id=67318')

,ssn,annual_income,id,name,license_id,address_number,address_street_name,ssn
0,100009868,52200,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
1,100169584,64500,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,100300433,74400,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
3,100355733,35900,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
4,100366269,73000,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
...,...,...,...,...,...,...,...,...
7509,999679296,54400,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
7510,999762859,77000,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
7511,999824984,82000,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
7512,999910617,82600,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [ ]:
#El asesino esJoe Germuska  con person id 28819 porque no tiene coartada de ese día